<a href="https://colab.research.google.com/github/lordbvico/msci-641-final-project/blob/main/Copy_of_deberta_training_task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
import pandas as pd
id = "1AYls0rBsSfK9HsWWj8kdN6prEJcZsFEk"
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('train.jsonl')
#df = pd.read_json('train.jsonl', lines=True)

In [ ]:
id1 = "18yu4xEsU1W_DYlT4dE6fCeXIvFJ_JSWQ"
downloaded1 = drive.CreateFile({'id':id1})
downloaded1.GetContentFile('val.jsonl')
#df2 = pd.read_json('val.jsonl', lines=True)

In [ ]:
id2 = "1z3X-ZrhbwIiabENW69nBUmj8O0Mb-_Er"
downloaded2 = drive.CreateFile({'id':id2})
downloaded2.GetContentFile('test.jsonl')
#df3 = pd.read_json('test.jsonl', lines=True)

In [ ]:
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 28.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.4 MB/s eta 0:00:00


In [ ]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.1 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.translate import meteor_score
import re
import json
from tqdm import tqdm
import copy
import wandb


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
sweep_config = {
        "name": "sweep",
        "method": "bayes",
        "metric": {"name": "Validation Accuracy", "goal": "maximize"},
        "parameters": {
            'learning_rate': {'max': 0.00001, 'min': 0.000001},
             "num_epochs": {"values": [3,4,5]}


        }
    }

    # Initialize Wandb sweep
sweep_id = wandb.sweep(sweep=sweep_config, project="T5 Large Parameter Sweep Aug 2 Task 2")

Create sweep with ID: sk2qp2kp
Sweep URL: https://wandb.ai/team_vico/T5%20Large%20Parameter%20Sweep%20Aug%202%20Task%202/sweeps/sk2qp2kp


In [ ]:
def remove_special_characters(text):

    # Convert the input to a string, regardless of its original type
    text = str(text)
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    # Substitute any non-word character with a space
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)


    cleaned_text = ' '.join(text.split())

    return cleaned_text


In [ ]:
def load_data(file_path):
    with open(file_path, 'r') as file:
        jsonl_content = file.readlines()

    data = []
    for json_str in jsonl_content:
        result = json.loads(json_str)
        result["postText"] = remove_special_characters(result["postText"])
        result["targetTitle"] = remove_special_characters(result["targetTitle"])
        result["targetParagraphs"] =  remove_special_characters(result["targetParagraphs"])
        result["spoiler"] = remove_special_characters(result["spoiler"])
        data.append(result)

    return data

def preprocess_data(data):
    preprocessed_data = []

    for row in data:
        # Convert targetParagraphs list to string if it's a list



        target_paragraphs = ' '.join(row['targetParagraphs']) if isinstance(row['targetParagraphs'], list) else row['targetParagraphs']
        input_str = f"generate_spoiler: {row['postText']} {row['targetTitle']} {target_paragraphs}"
        #input_str = f"generate_spoiler: {row['postText']} {row['targetTitle']} {target_paragraphs}"
        spoiler = row['spoiler']

        preprocessed_data.append({
            'input_str': input_str,
            'spoiler': spoiler
        })

    return preprocessed_data


In [ ]:
def load_data1(file_path):
    with open(file_path, 'r') as file:
        jsonl_content = file.readlines()

    data = []
    for json_str in jsonl_content:
        result = json.loads(json_str)
        result["postText"] = remove_special_characters(result["postText"])
        result["targetTitle"] = remove_special_characters(result["targetTitle"])
        result["targetParagraphs"] =  remove_special_characters(result["targetParagraphs"])
        data.append(result)

    return data

In [ ]:
def preprocess_test_data1(data):
    preprocessed_data = []

    for row in data:
        # Convert targetParagraphs list to string if it's a list
        target_paragraphs = ' '.join(row['targetParagraphs']) if isinstance(row['targetParagraphs'], list) else row['targetParagraphs']

        input_str = f"generate_spoiler: {row['postText']} {row['targetTitle']} {target_paragraphs}"
        def preprocess_test_data1(data):
    preprocessed_data = []

    for row in data:
        # Convert targetParagraphs list to string if it's a list
        target_paragraphs = ' '.join(row['targetParagraphs']) if isinstance(row['targetParagraphs'], list) else row['targetParagraphs']

        #input_str = f"generate_spoiler: {row['postText']} {row['targetTitle']} {target_paragraphs}"
        input_str = f"question: {row['postText']} context: {row['targetTitle']} {target_paragraphs}"

        preprocessed_data.append({
            'input_str': input_str
        })

    return preprocessed_data



        preprocessed_data.append({
            'input_str': input_str
        })

    return preprocessed_data



In [ ]:
train_data = load_data('train.jsonl')
val_data = load_data('val.jsonl')
test_data = load_data1('test.jsonl')
# Preprocess train data
train_data = preprocess_data(train_data)
val_data = preprocess_data(val_data)
test_data = preprocess_test_data1(test_data)
#train_data = train_data[:100]  # Use the first 1000 samples
#val_data = val_data[:20]  # Use the first 200 samples

In [ ]:
class ClickBaitDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data[index]
        inputs = self.tokenizer.encode_plus(
            row['input_str'],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        outputs = self.tokenizer.encode(
            row["spoiler"],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs["input_ids"].flatten(),
            "attention_mask": inputs["attention_mask"].flatten(),
            "labels": outputs.flatten(),
        }

In [ ]:
class ClickBaitTestDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data[index]
        inputs = self.tokenizer.encode_plus(
            row['input_str'],
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        return {
            "input_ids": inputs["input_ids"].flatten(),
            "attention_mask": inputs["attention_mask"].flatten(),
        }


In [ ]:
tokenizer = T5Tokenizer.from_pretrained("t5-large")
model = T5ForConditionalGeneration.from_pretrained("t5-large")

# replace train_data and val_data with your own data
train_dataset = ClickBaitDataset(train_data, tokenizer, max_length=512)
val_dataset = ClickBaitDataset(val_data, tokenizer, max_length=512)

train_loader = DataLoader(train_dataset, batch_size=4)
val_loader = DataLoader(val_dataset, batch_size=4)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:199: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available

In [ ]:
def main():
  run = wandb.init()
  learning_rate = wandb.config.learning_rate
  num_epochs = wandb.config.num_epochs

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  model.to(device)

  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  #best_val_loss = float('inf')
  best_val_accuracy = 0
  best_model = None

  #NUM_EPOCHS = 5

  # Training Loop
  for epoch in range(num_epochs):
      model.train()  # Set the model to training mode
      #for i, batch in enumerate(train_loader):
      for i, batch in tqdm(enumerate(train_loader), total=len(train_loader), desc="Training"):

          input_ids = batch["input_ids"].to(device)
          attention_mask = batch["attention_mask"].to(device)
          labels = batch["labels"].to(device)

          outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

          loss = outputs.loss
          loss.backward()

          optimizer.step()
          optimizer.zero_grad()

          if i % 100 == 0:
              print(f"Epoch: {epoch}, Loss:  {loss.item()}")

      model.eval()  # Set the model to evaluation mode
      val_loss = 0
      val_accuracy = 0
      with torch.no_grad():
          #for i, batch in enumerate(val_loader):
          for i, batch in tqdm(enumerate(val_loader), total=len(val_loader), desc="Validating"):
              input_ids = batch["input_ids"].to(device)
              attention_mask = batch["attention_mask"].to(device)
              labels = batch["labels"].to(device)


              # Compute loss for this batch
              loss_outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
              val_loss += loss_outputs.loss.item()

              # Generate predictions for this batch
              outputs = model.generate(input_ids, attention_mask=attention_mask)
              # Convert tensor outputs to strings
              outputs = outputs.cpu()

              # Decode the output and the label to text
              generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
              labels_text = tokenizer.batch_decode(labels, skip_special_tokens=True)

              # Calculate the METEOR score for each output and label pair, and sum them up
              batch_score = 0


              for ref, hyp in zip(labels_text, generated_text):
                  ref_tokens = ref.split()  # Tokenize reference sentence
                  hyp_tokens = hyp.split()  # Tokenize hypothesis sentence
                  batch_score += meteor_score.single_meteor_score(ref_tokens, hyp_tokens)
              batch_score /= len(labels_text)  # Calculate average METEOR score for this batch
              val_accuracy += batch_score
              #val_loss += outputs.loss.item()

      # Calculate the average validation loss
      val_loss = val_loss / len(val_loader)
      print(f"Validation Loss: {val_loss}")

      # Calculate the average METEOR score
      val_accuracy = val_accuracy / len(val_loader)
      print(f"Validation METEOR Score: {val_accuracy}")

      # Save the best model
      if val_accuracy > best_val_accuracy:
          best_val_accuracy = val_accuracy
          best_model = copy.deepcopy(model)
          model.save_pretrained(f"t5_spoiler_generation_model_lr{learning_rate}_accuracy{best_val_accuracy:.2f}_epoch{num_epochs}")
      wandb.log({"Validation Accuracy": val_accuracy, "Validation Loss": val_loss, "Learning Rate": learning_rate, "Number of Epochs": num_epochs})
      print(best_val_accuracy)

if __name__ == "__main__":
  wandb.agent(sweep_id, function=main)


wandb: Agent Starting Run: 0ukazmva with config:
wandb: 	learning_rate: 4.389723975538331e-06
wandb: 	num_epochs: 5
wandb: Currently logged in as: lordbvico (team_vico). Use `wandb login --relogin` to force relogin


Training:   0%|          | 1/800 [00:03<52:59,  3.98s/it]

Epoch: 0, Loss:  35.17081069946289


Training:  13%|█▎        | 101/800 [01:23<09:22,  1.24it/s]

Epoch: 0, Loss:  26.38850975036621


...

Epoch: 3, Loss:  0.008730911649763584


Validating: 100%|██████████| 100/100 [02:16<00:00,  1.37s/it]


Validation Loss: 0.03771935596363619
Validation METEOR Score: 0.33352353332744644
0.33352353332744644


Learning Rate,▁▁▁▁
Number of Epochs,▁▁▁▁
Validation Accuracy,▁▂▄█
Validation Loss,█▅▂▁
Learning Rate,1e-05
Number of Epochs,4
Validation Accuracy,0.33352
Validation Loss,0.03772


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
#best_model_path = f"t5_spoiler_generation_model_best_lr_{learning_rate}_epoch{num_epochs}"
best_model_path = f"t5_spoiler_generation_model_lr8.252438953448873e-06_accuracy0.31_epoch4"
best_model = T5ForConditionalGeneration.from_pretrained(best_model_path)
#/content/t5_spoiler_generation_model_best_lr_2.5812780043687333e-06_epoch5
#/content/t5_spoiler_generation_model_best_lr_7.728040301416214e-06_epoch4
#/content/t5_spoiler_generation_model_best_lr_6.601001159736487e-06_epoch4
#/content/t5_spoiler_generation_model_best_lr_8.316230405993195e-06_epoch3
#/content/t5_spoiler_generation_model_best_lr_9.415651315383846e-06_epoch3
#/content/t5_spoiler_generation_model_best_lr_6.777947478792724e-06_epoch7
#/content/t5_spoiler_generation_model_lr8.252438953448873e-06_accuracy0.33_epoch4
#/content/t5_spoiler_generation_model_lr8.252438953448873e-06_accuracy0.31_epoch4
test_dataset = ClickBaitTestDataset(test_data, tokenizer, max_length=512)
test_loader = DataLoader(test_dataset, batch_size=4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
best_model.to(device)
best_model.eval()  # Set the model to evaluation mode
generated_spoilers = []

with torch.no_grad():  # No need to track gradients
  for i, batch in enumerate(test_loader):


    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)

    # Generate spoilers
    outputs = best_model.generate(input_ids, attention_mask=attention_mask)


    # Decode the outputs into text
    outputs = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in outputs]


    generated_spoilers.extend(outputs)

# Create a DataFrame from the generated spoilers
df = pd.DataFrame(generated_spoilers, columns=['spoiler'])

# Add an index column
df.reset_index(level=0, inplace=True)
df = df.rename(columns = {'index':'id'})

# Save the DataFrame to a CSV file
#filename = f"generated_spoilers_lr_{learning_rate}_epochs_{num_epochs}_jul31.csv"
#df.to_csv(filename, index=False)
df.to_csv('generated_spoilers_lr8.252438953448873e-06_accuracy0.31_epoch4', index=False)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
